<a href="https://colab.research.google.com/github/hcorbel/fine_tuning_bert_stackoverflow/blob/main/stackoverflow_fine_tuning_bert__for_multi_label_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we are going to fine-tune a bert-base-uncased model to predict one or more labels for a given piece of text. This notebook is from https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
The model adds a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.

Dataset is from stackoverflow questions: body, title and tags that have been preprocessed. Only questions labelled by the 100 more frequent tags are kept (df_final.csv).

In [1]:
from google.colab import drive


In [2]:
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
import pandas as pd


In [4]:
df3=pd.read_csv("/gdrive/MyDrive/df_final.csv", sep=',')

In [5]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46732 entries, 0 to 46731
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         46732 non-null  int64  
 1   Title              46732 non-null  object 
 2   Body               46732 non-null  object 
 3   Tags               46732 non-null  object 
 4   Id                 46732 non-null  int64  
 5   Score              46732 non-null  int64  
 6   ViewCount          46732 non-null  int64  
 7   FavoriteCount      40590 non-null  float64
 8   AnswerCount        46732 non-null  int64  
 9   concat             46732 non-null  object 
 10  clean_tagB         46732 non-null  object 
 11  clean_tag          46732 non-null  object 
 12  clean_tag_list     46732 non-null  object 
 13  clean_concat       46732 non-null  object 
 14  clean_concat_list  46732 non-null  object 
 15  nb_tags            46732 non-null  int64  
 16  Tags_list2         467

In [6]:
df3=df3.sample(5)
df3['Tags_list2']

35497                                            ['linux']
40904    ['java', 'spring', 'hibernate', 'maven', 'spri...
23957                                 ['java', 'security']
39406                                              ['xml']
20226                                     ['c++', 'c++11']
Name: Tags_list2, dtype: object

In [7]:
df3.sample(5)


,Unnamed: 0,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount,concat,clean_tagB,clean_tag,clean_tag_list,clean_concat,clean_concat_list,nb_tags,Tags_list2,number_of_tags
40904,43783,Injection of autowired dependencies failed; ne...,<p>I am new here and I am trying to create an ...,<java><spring><hibernate><maven><spring-mvc>,42832387,6,12110,NaN,4,Injection of autowired dependencies failed; ne...,"['java', 'spring', 'hibernate', 'maven', 'spri...",java spring hibernate maven spring-mvc,"['java', 'spring', 'hibernate', 'maven', 'spri...",injection autowired dependency failed nested e...,"['injection', 'autowired', 'dependency', 'fail...",5,"['java', 'spring', 'hibernate', 'maven', 'spri...",5
39406,42189,Defining variables in SVG,<p>I currently have a line filled in SVG like ...,<xml><variables><path><svg><definition>,6556656,6,3066,0.0,1,Defining variables in SVG <p>I currently have ...,"['xml', 'variable', 'path', 'svg', 'definition']",xml variable path svg definition,"['xml', 'variable', 'path', 'svg', 'definition']",defining variable svg currently line filled sv...,"['defining', 'variable', 'svg', 'currently', '...",5,['xml'],1
20226,21686,How to make generic computations over heteroge...,<p><strong>PREMISE:</strong></p>\n\n<p>After p...,<c++><c++11><iteration><variadic-templates><te...,14261183,77,7300,0.0,5,How to make generic computations over heteroge...,"['c++', 'c++11', 'iteration', 'variadic-templa...",c++ c++11 iteration variadic-templates templat...,"['c++', 'c++11', 'iteration', 'variadic-templa...",generic computation heterogeneous argument pac...,"['generic', 'computation', 'heterogeneous', 'a...",5,"['c++', 'c++11']",2
23957,25660,Glassfish 3 security - Form based authenticati...,<p>I want to understand form based security a...,<java><security><jsf><jakarta-ee><glassfish>,7944963,24,26274,0.0,4,Glassfish 3 security - Form based authenticati...,"['java', 'security', 'jsf', 'jakarta-ee', 'gla...",java security jsf jakarta-ee glassfish,"['java', 'security', 'jsf', 'jakarta-ee', 'gla...",glassfish security form based authentication j...,"['glassfish', 'security', 'form', 'based', 'au...",5,"['java', 'security']",2
35497,38016,Yocto Bitbake - Git Fetch always stuck,<p>I am using YOCTO to build an image but the ...,<linux><linux-kernel><yocto><bitbake><openembe...,36850974,6,2974,NaN,3,Yocto Bitbake - Git Fetch always stuck <p>I am...,"['linux', 'linux-kernel', 'yocto', 'bitbake', ...",linux linux-kernel yocto bitbake openembedded,"['linux', 'linux-kernel', 'yocto', 'bitbake', ...",yocto bitbake git fetch always stuck yocto bui...,"['yocto', 'bitbake', 'git', 'fetch', 'always',...",5,['linux'],1


DATASET PREPARATION

On va créer un jeu d'entrainement, de test et de validation. On a 1 colonne par tag (n=100 colonnes + 1 colonne de texte = [corps + titre]) à partir de 10000 questions sélectionnées au hasard dans le dataaset

In [ ]:
df3=df3.sample(10000)
df3=df3[['Tags_list2', 'clean_concat']]

In [ ]:
df3.head()

,Tags_list2,clean_concat
24477,iphone image,io icloud photo transfer ability new icloud se...
22726,c++,collada inverse bind pose wrong space writing ...
18391,python,set tab notebook far widget unable set tab kee...
314,python authentication,passing apache2 digest authentication informat...
23181,java spring security spring-boot,securing spring data repositoryrestresource cr...


In [ ]:
df3['Tags_list2']=df3['Tags_list2'].str.replace("[","")
df3['Tags_list2']=df3['Tags_list2'].str.replace("]","")
df3['Tags_list2']=df3['Tags_list2'].str.replace("'","")
df3['Tags_list2']=df3['Tags_list2'].str.replace(",","")
corpustag=" ".join(df3.Tags_list2.values)
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_tag = CountVectorizer (tokenizer = lambda x: x.split())
tag_tf = vectorizer_tag.fit_transform([corpustag])
print("Number of tags :", tag_tf.shape[1])
count_array = tag_tf.toarray()
b = pd.DataFrame(data=count_array,columns = vectorizer_tag.get_feature_names_out())
print(b)


Number of tags : 100
   .net  ajax  algorithm  amazon-web-services  android  android-studio  \
0   681   112        132                  102      719              71   

   angular  angularjs  array  asp.net  ...  ubuntu  unit-testing  \
0      116        106    158      295  ...      74           164   

   visual-studio  web-services  window  winforms  wpf  xaml  xcode  xml  
0            164            66     217        89  155    67    233  111  

[1 rows x 100 columns]


<ipython-input-298-f79f7998f8ba>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df3['Tags_list2']=df3['Tags_list2'].str.replace("[","")
<ipython-input-298-f79f7998f8ba>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df3['Tags_list2']=df3['Tags_list2'].str.replace("]","")
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
df4=df3
df4['Tags_list2']=df4['Tags_list2'].str.replace("[","")
df4['Tags_list2']=df4['Tags_list2'].str.replace("]","")
df4['Tags_list2']=df4['Tags_list2'].str.replace("'","")
df4['Tags_list2']=df4['Tags_list2'].str.replace(",","")
mlb = MultiLabelBinarizer()
df5 = (pd.DataFrame(mlb.fit_transform(df4['Tags_list2'].str.split()),
                    columns=mlb.classes_, index=df4['clean_concat']).eq(1))
df5.head()

<ipython-input-299-aced475fdd97>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df4['Tags_list2']=df4['Tags_list2'].str.replace("[","")
<ipython-input-299-aced475fdd97>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df4['Tags_list2']=df4['Tags_list2'].str.replace("]","")


,.net,ajax,algorithm,amazon-web-services,android,android-studio,angular,angularjs,array,asp.net,...,ubuntu,unit-testing,visual-studio,web-services,window,winforms,wpf,xaml,xcode,xml
clean_concat,,,,,,,,,,,,,,,,,,,,,
io icloud photo transfer ability new icloud service many configuration may device configured send picture icloud server storing device,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
collada inverse bind pose wrong space writing collada importer ive pretty far loading mesh material ive hit snag animation specifically joint rotation formula skinning mesh straight-forward far literature concerned correct formula collada specifies type rotation joint local global concatenate rotation local transformation joint collada documentation differentiate joint local rotation joint global rotation model ive seen rotation id either global local disregard global rotation local bind pose model add global rotation joint local transformation strange start happen global rotation global rotation screenshots drawing skeleton line invisible joint inside mesh screenshot vertex place comparison screenshot look hard see see joint correct position screenshot weird disregard inverse bind pose specified collada inverse joint parent local transform time joint local transform screenshot drawing line vertex joint influence fact bind pose strange formula becomes lead suspect colladas inverse bind pose wrong space space collada specifies inverse bind pose transform inverse bind pose space,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
set tab notebook far widget unable set tab keep piling eastward set stack somehow,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
passing apache2 digest authentication information wsgi script run modwsgi ive directive id somescript.wsgi logged,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
securing spring data repositoryrestresource crudrepository http internally spring data project us repositoryrestresource crudrepositorys expose entity rest able secure repository accessed http secured used internally service layer example spring security running adding annotation preauthorize crudrepository method cause security expression executed method service layer id glad someone point right direction edit ive removing rest export security annotation userrepository internally subclassing userrepository userrepositoryrestexported exporting securing seeing inconsistent implementation security annotation run wonder spring exporting userrepositoryrestexported sometimes userrepository times... edit code described edit userrepository.java userrepositoryrest.java,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
df5.reset_index(inplace=True)
df5.head()

,clean_concat,.net,ajax,algorithm,amazon-web-services,android,android-studio,angular,angularjs,array,...,ubuntu,unit-testing,visual-studio,web-services,window,winforms,wpf,xaml,xcode,xml
0,io icloud photo transfer ability new icloud se...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,collada inverse bind pose wrong space writing ...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,set tab notebook far widget unable set tab kee...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,passing apache2 digest authentication informat...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,securing spring data repositoryrestresource cr...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
path = '/gdrive/My Drive/df6.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df5.to_csv(f, index=False)

In [ ]:
df5read=pd.read_csv("/gdrive/MyDrive/df6.csv", sep=',')
df5read.head()

,clean_concat,.net,ajax,algorithm,amazon-web-services,android,android-studio,angular,angularjs,array,...,ubuntu,unit-testing,visual-studio,web-services,window,winforms,wpf,xaml,xcode,xml
0,curl httpurlconnection post json data post jso...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,double.parse internationalization problem driv...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,composite location screen implementing screen ...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,awsecs cpuutilization average v maximum readin...,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,usage let assignment swift swift documentation...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
!pip install -q transformers datasets

In [ ]:
from datasets import *
dataset = load_dataset('csv', data_files="/gdrive/MyDrive/df6.csv")

train_testvalid = dataset['train'].train_test_split(test_size=0.2)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2d8cae3f92922cb2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_concat', '.net', 'ajax', 'algorithm', 'amazon-web-services', 'android', 'android-studio', 'angular', 'angularjs', 'array', 'asp.net', 'asp.net-core', 'asp.net-mvc', 'assembly', 'asynchronous', 'authentication', 'bash', 'c', 'c#', 'c++', 'c++11', 'class', 'cocoa', 'cocoa-touch', 'cs', 'database', 'debugging', 'django', 'docker', 'eclipse', 'entity-framework', 'file', 'firebase', 'gcc', 'git', 'google-chrome', 'gradle', 'hibernate', 'html', 'http', 'image', 'io', 'ios7', 'iphone', 'java', 'javascript', 'jpa', 'jquery', 'json', 'laravel', 'linq', 'linux', 'list', 'machine-learning', 'macos', 'maven', 'memory', 'multithreading', 'mysql', 'node.js', 'numpy', 'objective-c', 'optimization', 'panda', 'performance', 'php', 'postgresql', 'python', 'python-2.7', 'python-3.x', 'qt', 'r', 'reactjs', 'regex', 'rest', 'ruby', 'ruby-on-rails', 'scala', 'security', 'shell', 'spring', 'spring-boot', 'spring-mvc', 'sql', 'sql-server', 'ssl', 's

In [ ]:
example = dataset['train'][0]
example

{'clean_concat': 'rail document.ready turbo-links ran rail app organize j file rail previously scattered across different view organized separate file compile asset pipeline learned jquerys ready event fire subsequent click turbo-linking turned time load page click link anything inside wont executed page actually load explanation right ensure jquery event properly turbo-links wrap script rails-specific listener maybe rail magic unnecessary doc bit vague especially respect loading multiple file manifest application.js',
 '.net': False,
 'ajax': False,
 'algorithm': False,
 'amazon-web-services': False,
 'android': False,
 'android-studio': False,
 'angular': False,
 'angularjs': False,
 'array': False,
 'asp.net': False,
 'asp.net-core': False,
 'asp.net-mvc': False,
 'assembly': False,
 'asynchronous': False,
 'authentication': False,
 'bash': False,
 'c': False,
 'c#': False,
 'c++': False,
 'c++11': False,
 'class': False,
 'cocoa': False,
 'cocoa-touch': False,
 'cs': False,
 'datab

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['clean_concat']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['.net',
 'ajax',
 'algorithm',
 'amazon-web-services',
 'android',
 'android-studio',
 'angular',
 'angularjs',
 'array',
 'asp.net',
 'asp.net-core',
 'asp.net-mvc',
 'assembly',
 'asynchronous',
 'authentication',
 'bash',
 'c',
 'c#',
 'c++',
 'c++11',
 'class',
 'cocoa',
 'cocoa-touch',
 'cs',
 'database',
 'debugging',
 'django',
 'docker',
 'eclipse',
 'entity-framework',
 'file',
 'firebase',
 'gcc',
 'git',
 'google-chrome',
 'gradle',
 'hibernate',
 'html',
 'http',
 'image',
 'io',
 'ios7',
 'iphone',
 'java',
 'javascript',
 'jpa',
 'jquery',
 'json',
 'laravel',
 'linq',
 'linux',
 'list',
 'machine-learning',
 'macos',
 'maven',
 'memory',
 'multithreading',
 'mysql',
 'node.js',
 'numpy',
 'objective-c',
 'optimization',
 'panda',
 'performance',
 'php',
 'postgresql',
 'python',
 'python-2.7',
 'python-3.x',
 'qt',
 'r',
 'reactjs',
 'regex',
 'rest',
 'ruby',
 'ruby-on-rails',
 'scala',
 'security',
 'shell',
 'spring',
 'spring-boot',
 'spring-mvc',
 'sql',
 'sql-serv

PREPROCESSING

As models like BERT don't expect text as direct input, but rather input_ids, etc., we tokenize the text using the tokenizer. Here I'm using the AutoTokenizer API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' BCEWithLogitsLoss (which the model will use) will complain.

D'abord, on configure l'environnement d'execution

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16068677739616307461
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 40146370560
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9172724579369636486
 physical_device_desc: "device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
 xla_global_id: 416903419]

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.test.is_gpu_available()

True

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr 27 23:54:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    50W / 400W |    691MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["clean_concat"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] rail document. ready turbo - links ran rail app organize j file rail previously scattered across different view organized separate file compile asset pipeline learned jquerys ready event fire subsequent click turbo - linking turned time load page click link anything inside wont executed page actually load explanation right ensure jquery event properly turbo - links wrap script rails - specific listener maybe rail magic unnecessary doc bit vague especially respect loading multiple file manifest application. js [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['javascript', 'jquery']

In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch datasets.

In [ ]:
encoded_dataset.set_format("torch")

MODELE DEFINING

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the problem_type to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely BCEWithLogitsLoss). We also make sure the output layer has len(labels) output neurons, and we set the id2label and label2id mappings

In [ ]:
from transformers import AutoModelForSequenceClassification
device = torch.device('cuda')

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
TRAINING

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

1. TrainingArguments, which specify training hyperparameters. Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
2. a Trainer object.

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert_fine_tune",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a compute_metrics function, that returns a dictionary with the desired metric values.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.3):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [ ]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  4334,  6254,  1012,  3201, 15386,  1011,  6971,  2743,  4334,
        10439, 10939,  1046,  5371,  4334,  3130,  7932,  2408,  2367,  3193,
         4114,  3584,  5371,  4012, 22090, 11412, 13117,  4342,  1046,  4226,
        24769,  3201,  2724,  2543,  4745, 11562, 15386,  1011, 11383,  2357,
         2051,  7170,  3931, 11562,  4957,  2505,  2503,  2180,  2102,  6472,
         3931,  2941,  7170,  7526,  2157,  5676,  1046,  4226,  2854,  2724,
         7919, 15386,  1011,  6971, 10236,  5896, 15168,  1011,  3563, 19373,
         2672,  4334,  3894, 14203,  9986,  2978, 13727,  2926,  4847, 10578,
         3674,  5371, 19676,  4646,  1012,  1046,  2015,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
#forward pass

outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0).to(device), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0).to(device))
outputs

SequenceClassifierOutput(loss=tensor(0.6937, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 4.7180e-01, -2.1380e-01,  2.7178e-01, -1.3833e-01,  2.3408e-01,
         -3.3622e-01, -3.0687e-01, -2.3369e-01,  3.2611e-01, -3.5674e-01,
         -1.4725e-01,  1.0937e-04,  8.9060e-02, -2.6770e-02, -3.2646e-02,
         -2.3940e-01,  2.2528e-01, -9.5567e-02,  1.9416e-01,  2.3052e-01,
          2.1283e-01, -3.4994e-01,  5.7610e-01,  6.0834e-02,  2.1432e-01,
          7.2628e-02, -1.2883e-01,  3.6646e-02, -4.2500e-01,  2.5927e-01,
         -1.1636e-01,  1.5715e-01, -1.2680e-01,  1.0249e-01, -1.3801e-02,
         -2.4396e-01,  7.0200e-04,  3.0013e-01,  5.4438e-02, -5.1050e-01,
         -1.1927e-01, -1.9443e-01,  7.4794e-02,  1.2441e-01, -2.2036e-01,
          2.3262e-01, -1.4330e-01, -7.0720e-02, -8.1820e-01, -4.0913e-02,
         -1.1963e-01, -3.9059e-02, -6.3964e-02, -9.6202e-02, -1.9908e-01,
         -8.8994e-02,  3.2777e-01, -2.3218e-01, -9.4811e-02,

we start training:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
a=trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.093700,0.090306,0.000000,0.500000,0.000000
2,0.087700,0.083082,0.121048,0.532694,0.028000
3,0.074800,0.072042,0.395150,0.635021,0.114000
4,0.068400,0.066547,0.438025,0.658716,0.138000


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.093700,0.090306,0.000000,0.500000,0.000000
2,0.087700,0.083082,0.121048,0.532694,0.028000
3,0.074800,0.072042,0.395150,0.635021,0.114000
4,0.068400,0.066547,0.438025,0.658716,0.138000
5,0.064600,0.064888,0.444815,0.662366,0.143000


EVALUATION

After the training, we evaluate our model on the validation set

In [ ]:
trainer.evaluate()

{'eval_loss': 0.06488791108131409,
 'eval_f1': 0.44481493831277097,
 'eval_roc_auc': 0.6623662163855091,
 'eval_accuracy': 0.143,
 'eval_runtime': 7.6888,
 'eval_samples_per_second': 130.06,
 'eval_steps_per_second': 16.258,
 'epoch': 5.0}

INFERENCE

we test the model on an exemple from the dataset

In [ ]:
text = "securing spring data repositoryrestresource crudrepository http internally spring data project us repositoryrestresource crudrepositorys expose entity rest able secure repository accessed http secured used internally service layer example spring security running adding annotation preauthorize crudrepository method cause security expression executed method service layer id glad someone point right direction edit ive removing rest export security annotation userrepository internally subclassing userrepository userrepositoryrestexported exporting securing seeing inconsistent implementation security annotation run wonder spring exporting userrepositoryrestexported sometimes userrepository times... edit code described edit userrepository.java userrepositoryrest.java"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 100])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example). The chosen threshold here is very low because of the very high number of labels.

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
#ou bien sélectionner les 5 premiers tags (ici si p > 0.08)
predictions[np.where(probs >= 0.08)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)
##reel  = java spring security spring-boot

['android', 'java', 'spring', 'spring-boot', 'spring-mvc']
